### Single transmon optimisation
Single transmon with five energy levels. The lowest four levels are being used for storing two qubits. The generation creates a superposition of 2 LOs and envelopes. Single- and two-qubit gates can be optimised.

In [1]:
import os, sys, argparse
from collections import OrderedDict

# Main C3 objects
import numpy as np

from c3.libraries import constants
from c3.parametermap import ParameterMap as PMap
from c3.experiment import Experiment as Exp
from c3.model import Model as Mdl
from c3.utils.tf_utils import tf_project_to_comp

# Building blocks
import c3.generator.devices as devices
import c3.signal.gates as gates
import c3.libraries.chip as chip
import c3.signal.pulse as pulse
import c3.libraries.tasks as tasks

# Libs and helpers
import c3.libraries.algorithms as algorithms
import c3.libraries.fidelities as fidelities
import c3.utils.qt_utils as qt_utils
from c3.optimizers.optimalcontrol import OptimalControl

#%matplotlib widget

from four_level_transmons.utilities import *
from four_level_transmons.plotting import *
from four_level_transmons.custom_envelopes import *
from four_level_transmons.DataOutput import DataOutput
from four_level_transmons.notebook_utils import *

import four_level_transmons.custom_gates as custom_gates
from c3.utils import tf_utils
tf.config.run_functions_eagerly(True)
tf.get_logger().setLevel('ERROR')
np.set_printoptions(linewidth=300)

2023-10-24 13:41:25.827656: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-24 13:41:25.827706: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
if len(sys.argv[1:]) > 0 and "ipykernel_launcher" not in sys.argv[0]:
    parser = argparse.ArgumentParser()
    parser.add_argument("--output", help="Output directory")
    args = parser.parse_args()
    output_dir = args.output
    print("Output directory: ", output_dir)
else:
    print("=========== WARNING: no output directory specified ============")
    output_dir = "./output"

=========== WARNING: no output directory specified ============


In [3]:
numPWCPieces = 60
usePWC = False
useDRAG = True
t_final = 40e-9 #1e-11
sim_res = 100e9 #1000e9
awg_res = numPWCPieces / t_final if usePWC else 20e9 #1000e9
USE_COMPLEX_GAUSSIANS = False
USE_COMPLEX_AMPLITUDE = False

#ideal_gate, ideal_gate_name = qt_utils.np_kron_n([constants.GATES["rx90p"], constants.Id]), "rx90p_q1"
ideal_gate, ideal_gate_name = qt_utils.np_kron_n([constants.Id, constants.GATES["rz90p"]]), "rz90p_q2"
#ideal_gate, ideal_gate_name = qt_utils.np_kron_n([constants.Id, constants.H]), "Z_q1"
#ideal_gate, ideal_gate_name = np.array([[1, 0, True0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]]) / np.sqrt(2.0), "swap"
#ideal_gate, ideal_gate_name = np.array([[1,0,0,-1j], [0,1,-1j,0], [0,-1j,1,0], [-1j,0,0,1]]) / np.sqrt(2.0), "double_iswap"
#ideal_gate, ideal_gate_name = constants.ISWAP, "iswap"

INPUT_FILE = "./optimised_params/single transmon/2 LOs/rz90p_q2_40ns.json"

algorithm = algorithms.lbfgs
algorithmParameters = {"maxfun": 1000, "ftol": 1e-8}

#algorithm = algorithms.cmaes
#algorithmParameters = {"popsize" : 10, "maxfevals" : 300, "init_point" : "True", "tolfun" : 0.01, "spread" : 0.25 }

In [15]:
def run(qubit_frequency: float, anharmonicity: float, inputFile: str, increase: float) -> str:
    # Initialise the qubits and drive lines
    qubit_levels = 5

    t1 = 25e-6
    t2star = 35e-6
    qubit_temp = 50e-3
    output = DataOutput(output_dir, file_suffix='before')
    qubit = createQubits([qubit_levels], [qubit_frequency], [anharmonicity], [t1], [t2star], qubit_temp)[0]
    drive = createDrives([qubit])[0]

    # Create the model
    model = Mdl([qubit], [drive])
    model.set_lindbladian(False)
    model.set_dressed(False)
    model.set_FR(False)
    energies = getEnergiesFromHamiltonian(qubit.get_Hamiltonian())
    print("energies: ", energies)
    transitions = np.array([energies[i + 1] - energies[i] for i in range(len(energies) - 1)])
    print("transition frequencies: ", transitions)

    # Envelopes and carriers
    N = 2
    generator = createGeneratorNLOs([drive], sim_res=sim_res, awg_res=awg_res, N=N)
    #carrier_freqs = [transitions[0], transitions[2]]
    #print("Carrier frequencies: ", carrier_freqs)

    envelopes = []
    carriers = []
    stored_pmap = PMap()
    stored_pmap.read_config(inputFile)
    stored_params = stored_pmap.asdict()[ideal_gate_name + "[0]"]
    stored_params_d = stored_params["drive_channels"]["d1"]
    for i in range(N):
        env = stored_params_d[f"envelope_d1_{i + 1}"]
        env.params["t_final"] = Qty(value=t_final, min_val=0.8 * t_final, max_val=t_final, unit="s")

        if useDRAG and type(env) is not pulse.EnvelopeDrag:
            env = convertToDRAG(env)
        elif not useDRAG and type(env) is pulse.EnvelopeDrag:
            env = convertFromDRAG(env)
        envelopes.append(env)

        carrier = stored_params_d[f"carrier_d1_{i + 1}"]
        #oldValue = carrier.params['freq'].get_value().numpy() / (2 * np.pi)
        #newValue = oldValue + increase
        #print("Carrier: ", oldValue, '->', newValue)
        #carrier.params['freq'].set_value(newValue)
        #carrier.params['freq'] = Qty(value=carrier_freqs[i], min_val=0.8 * carrier_freqs[i],
        #                             max_val=1.2 * carrier_freqs[i],
        #                             unit=carrier.params['freq'].unit)
        #oldValue = carrier.params['freq'].get_value() / (2 * np.pi)
        #carrier.params['freq'].set_value(oldValue + 10e6)
        carriers.append(carrier)

    print("Carriers: ", [c.params["freq"] for c in carriers])
    print("Amps: ", [e.params["amp"] for e in envelopes])

    # Gate instructions
    #printMatrix(ideal_gate, createSingleTransmonQubitLabels(4, makeLatexKets=True), "ideal_gate", output)
    gate = gates.Instruction(
        name=ideal_gate_name,
        targets=[0],
        t_start=0.0,
        t_end=t_final,
        channels=[drive.name],
        ideal=ideal_gate,
    )
    for env in envelopes:
        gate.add_component(copy.deepcopy(env), drive.name)
    for carrier in carriers:
        gate.add_component(copy.deepcopy(carrier), drive.name)

    # Set up the experiment
    parameter_map = PMap(instructions=[gate], model=model, generator=generator)
    exp = Exp(pmap=parameter_map)
    exp.set_opt_gates([gate.get_key()])

    stateEnergies = getEnergiesFromHamiltonian(model.get_Hamiltonian())
    transitionLabels = calculateTransitions(stateEnergies, createSingleTransmonQubitLabels(5, makeLatexKets=True))
    generator.generate_signals(gate)
    #printSignal(exp, [qubit], gate, output=output, states=transitionLabels)

    unitaries = exp.compute_propagators()
    #printPropagator(exp, gate, createSingleTransmonQubitLabels(5, makeLatexKets=True, labelLeakageAsNumbers=False),
    #    createSingleTransmonQubitLabels(4, makeLatexKets=True), output)

    # Calculate and save the time-dependent Hamiltonian
    signal = parameter_map.generator.generate_signals(parameter_map.instructions[gate.get_key()])
    H = parameter_map.model.get_Hamiltonian(signal)
    output.save(H, 'hamiltonian_timedependent')

    initialFidelity = 1 - tf_utils.tf_unitary_overlap(tf_project_to_comp(
        unitaries[gate.get_key()], dims=[5], index=[0], outdims=[4]
    ), ideal_gate, lvls=4).numpy()
    print("Initial infidelity: ", initialFidelity)

    #parameter_map.write_config(output.createFileName('parameter_map', 'json'))

    # Specify the parameters to be optimised and initialise the optimiser
    opt_map = []
    for env in envelopes:
        if USE_COMPLEX_GAUSSIANS:
            opt_map.append([(gate.get_key(), "d1", env.name, "sigmaR")])
            opt_map.append([(gate.get_key(), "d1", env.name, "sigmaI")])
        else:
            opt_map.append([(gate.get_key(), "d1", env.name, "sigma")])
        if USE_COMPLEX_AMPLITUDE:
            opt_map.append([(gate.get_key(), "d1", env.name, "ampR")])
            opt_map.append([(gate.get_key(), "d1", env.name, "ampI")])
        else:
            opt_map.append([(gate.get_key(), "d1", env.name, "amp")])
        #opt_map.append([(gate.get_key(), "d1", env.name, "freq_offset")])
        opt_map.append([(gate.get_key(), "d1", env.name, "xy_angle")])
        #opt_map.append([(gate.get_key(), "d1", env.name, "t_final")])
        if useDRAG:
            opt_map.append([(gate.get_key(), "d1", env.name, "delta")])
    for carrier in carriers:
        opt_map.append([(gate.get_key(), "d1", carrier.name, "freq")])
    #    #opt_map.append([(gate.get_key(), "d1", carrier.name, "framechange")])
    parameter_map.set_opt_map(opt_map)
    parameter_map.print_parameters()

    infidelities = optimise(output, [qubit], exp, algorithm, algorithmParameters, gate)

    # Plot results
    output = DataOutput(output_dir, file_suffix='')
    #plotData(np.arange(len(infidelities)), infidelities, xlabel="Step",
    #         ylabel="Infidelity", filename=output.createFileName("convergence", "svg"))
    #printSignal(exp, [qubit], gate, output=output, states=transitionLabels)
    #printAllSignals(exp, qubit, output, directory="devices_after")
    #printPropagator(exp, gate, createSingleTransmonQubitLabels(5, makeLatexKets=True, labelLeakageAsNumbers=False),
    #    createSingleTransmonQubitLabels(4, makeLatexKets=True), output)
    filename = f'parameter_map_{anharmonicity}'
    parameter_map.write_config(output.createFileName(filename, 'json'))
    return filename

In [16]:
inputFile = INPUT_FILE
frequency = 5e9
increase = 1e5
while frequency <= 6e9:
    print('========================', frequency)
    output = run(frequency, -300e6, inputFile, increase)
    inputFile = './output/' + output + '.json'
    frequency += increase

======================== 5000000000.0
energies:  [0.00e+00 5.00e+09 9.70e+09 1.41e+10 1.82e+10]
transition frequencies:  [5.0e+09 4.7e+09 4.4e+09 4.1e+09]
Carrier:  4989585058.152732 -> 4989685058.152732
Carrier:  4388991867.696206 -> 4389091867.696206
Carriers:  [4.990 GHz 2pi, 4.389 GHz 2pi]
Amps:  [305.339 mV, 184.267 mV]
Initial infidelity:  [6.9227967e-05]
rz90p_q2[0]-d1-envelope_d1_1-sigma    : 8.045 ns 
rz90p_q2[0]-d1-envelope_d1_1-amp      : 305.339 mV 
rz90p_q2[0]-d1-envelope_d1_1-xy_angle : 6.123 mrad 
rz90p_q2[0]-d1-envelope_d1_1-delta    : -9.354 m 
rz90p_q2[0]-d1-envelope_d1_2-sigma    : 7.735 ns 
rz90p_q2[0]-d1-envelope_d1_2-amp      : 184.267 mV 
rz90p_q2[0]-d1-envelope_d1_2-xy_angle : -3.130 mrad 
rz90p_q2[0]-d1-envelope_d1_2-delta    : 10.245 m 

C3:STATUS:Saving as: /home/user/c3/output/rz90p_q2/2023_10_24_T_13_54_09/open_loop.c3log
1 6.922796698305422e-05 -6.922797e-05 (step time: 0.12708425521850586 sec) (average step time: 0.12708425521850586 sec)
2 0.5881652861323

KeyboardInterrupt: 